In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier


In [ ]:
# Cargar datos
exitosas = pd.read_csv('data/transacciones_exitosas.csv', low_memory=False)
fallidas = pd.read_csv('data/transacciones_fallidas.csv')
evaluacion = pd.read_csv('data/transacciones_evaluación.csv')

# Eliminar fila con mal formato
exitosas = exitosas.drop(148541)

# Añadir etiquetas a los datos
exitosas['label'] = 0
fallidas['label'] = 1

# Combinar datos exitosos y fallidos
datos = pd.concat([exitosas, fallidas], ignore_index=True)

# Separar características y etiquetas
X = datos.drop(columns=['label'])
y = datos['label']


In [ ]:
# División del conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Llenar valores NaN con 0 (u otro método como media o mediana de la columna)
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

# Escalar los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
# Definir el modelo y los parámetros para GridSearchCV
model = RandomForestClassifier()
params = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20],
}

# GridSearchCV para selección de modelos
grid = GridSearchCV(model, params, cv=5, scoring='f1')
grid.fit(X_train, y_train)
best_model = grid.best_estimator_


In [ ]:
# Evaluar el modelo
y_pred = best_model.predict(X_test)
f1 = f1_score(y_test, y_pred)
print(f'RandomForestClassifier: f1 Score = {f1}')


In [ ]:
# Combinar datos de entrenamiento y prueba
X_combined = np.vstack((X_train, X_test))
y_combined = np.concatenate((y_train, y_test))

# Entrenar el mejor modelo con todos los datos disponibles
final_model = best_model
final_model.fit(X_combined, y_combined)


In [ ]:
# Predecir probabilidades de falla en el conjunto de evaluación
X_evaluacion = evaluacion.drop(columns=['identificador'])  # Asegúrate de ajustar esto según tu archivo de evaluación
X_evaluacion = scaler.transform(X_evaluacion)
predicciones = final_model.predict_proba(X_evaluacion)[:, 1]

# Seleccionar las 200 transacciones con mayor probabilidad de falla
evaluacion['probabilidad_falla'] = predicciones
top_200 = evaluacion.nlargest(200, 'probabilidad_falla')['identificador']

# Guardar los identificadores de las 200 transacciones seleccionadas
top_200.to_csv('data/top_200_transacciones_fallidas.csv', index=False)
